In [2]:
print("Testing")

Testing


In [3]:
# Hate speech is generally a task of sentiment classi
# Can be achived by traing it by data


## HATE SPEECH DETECTION MODEL

In [5]:
import pandas as pd

In [8]:
train=pd.read_csv('train.csv')
print("Training Set :"%train.columns,train.shape,len(train))

Training Set : (31962, 3) 31962


In [11]:
test=pd.read_csv('test.csv')
print("Test Set :"%test.columns,test.shape,len(test))


Test Set : (17197, 2) 17197


In [16]:
import re
def clean_text(df,text_feild):
  df[text_feild]=df[text_feild].str.lower()
  df[text_feild] = df[text_feild].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))
  return df
test_clean = clean_text(test, "tweet")
train_clean = clean_text(train, "tweet")

#can use oversampling or down sampling
# (@[A-Za-z0-9] matches twitter handle
#([^0-9A-Za-z \t]): Matches any character that is not a digit, letter, space, or tab. This effectively removes special characters and punctuation.
#(\w+:\/\/\S+): Matches URLs starting with a word followed by :// and then any non-whitespace characters.
#^rt: Matches the string "rt" at the beginning of the text, which indicates a retweet in Twitter.
#http.+?: Matches any URL starting with "http".$

## Steps which are followed below for sampling
Import Resample: Use resample from sklearn.utils for upsampling.
Separate Majority Class: Create train_majority with rows where label == 0.
Separate Minority Class: Create train_minority with rows where label == 1.
Upsample Minority: Use resample to increase train_minority to the size of train_majority.
Combine Classes: Concatenate train_minority_upsampled and train_majority to form train_upsampled.
Verify Balance: Check the distribution of labels in train_upsampled using value_counts().

In [17]:
from sklearn.utils import resample
train_majority = train_clean[train_clean.label==0]
train_minority = train_clean[train_clean.label==1]
train_minority_upsampled = resample(train_minority,
                                 replace=True,
                                 n_samples=len(train_majority),
                                 random_state=123)
train_upsampled = pd.concat([train_minority_upsampled, train_majority])
train_upsampled['label'].value_counts()

label
1    29720
0    29720
Name: count, dtype: int64

## STEPS / BASIC EXPLAINATION FOR THE FOLLOWING CODE
Certainly! Here is the code explained point by point in one line each:

1. **Import TfidfVectorizer**: `from sklearn.feature_extraction.text import TfidfVectorizer` - Import for creating TF-IDF features.
2. **Import Pipeline**: `from sklearn.pipeline import Pipeline` - Import to create a machine learning pipeline.
3. **Import CountVectorizer**: `from sklearn.feature_extraction.text import CountVectorizer` - Import for converting text to a matrix of token counts.
4. **Import TfidfTransformer**: `from sklearn.feature_extraction.text import TfidfTransformer` - Import for transforming token counts to TF-IDF representation.
5. **Import SGDClassifier**: `from sklearn.linear_model import SGDClassifier` - Import for linear classifier optimized by stochastic gradient descent.
6. **Define Pipeline**: `pipeline_sgd = Pipeline([ ... ])` - Create a pipeline named `pipeline_sgd` that sequentially applies text transformations and a classifier.
7. **Step 1 - CountVectorizer**: `('vect', CountVectorizer())` - First step, tokenize text and convert to token count matrix.
8. **Step 2 - TfidfTransformer**: `('tfidf', TfidfTransformer())` - Second step, transform token counts to TF-IDF features.
9. **Step 3 - SGDClassifier**: `('nb', SGDClassifier())` - Third step, apply SGDClassifier to the TF-IDF features for training a linear model.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
pipeline_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),])

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_upsampled['tweet'],
                                                    train_upsampled['label'],random_state = 0)

In [20]:
model = pipeline_sgd.fit(X_train, y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import f1_score
f1_score(y_test, y_predict)

0.969729297239632